# Bootcamp DataScience iHub

## Gasto de deputados por gênero

### Etapas: 
    1. Abrir os datasets do [link]: http://dontpad.com/thaisviana/datascience
    2. Calcular o gasto por cada deputado em 2018
    3. Descobrir quem são os deputados em 2018 no nosso dataset
    4. Filtrar o dataset de informações de deputados pelos deputados na lista do item 3
    5. Criar um dataset novo com as colunas NOME, GENERO, GASTO
    6. Calcular qual genero gasta mais em média por número de agentes públicos

In [0]:
import pandas as pd
import numpy as np
from pandas.api.types import is_string_dtype


In [0]:
df_2018 = pd.read_csv('Ano-2018.csv', sep=';', parse_dates=['datEmissao'])

In [0]:
df_infos = pd.read_csv('deputados.csv', sep=';', parse_dates=['dataNascimento'])

## Gasto por cada deputado em 2018

In [0]:
df_2018 = df_2018[pd.isna(df_2018['sgPartido']) == False]
gasto = df_2018.groupby(['txNomeParlamentar', 'sgPartido'])['vlrDocumento'].sum()

##  Deputados em 2018 no nosso dataset

In [0]:
lista_de_nomes = df_2018.txNomeParlamentar.unique()

In [0]:
#gasto.index

## Filtrar o dataset de info de deputados pelos deputados na lista

In [14]:
df_nome_genero = df_infos[df_infos['nome'].isin(lista_de_nomes) == True]
df_nome_genero

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
221,https://dadosabertos.camara.leg.br/api/v2/depu...,MACEDO,3,3,José Vicente de Macedo,NaN,M,NaN,NaN,NaN,NaN,NaN,NaN
376,https://dadosabertos.camara.leg.br/api/v2/depu...,ROCHA,5,9,Justiniano José da Rocha,NaN,M,NaN,NaN,1812-11-08,1862-07-10,RJ,Rio de Janeiro
477,https://dadosabertos.camara.leg.br/api/v2/depu...,MACEDO,7,7,Marcos Antonio de Macedo,NaN,M,NaN,NaN,1808-06-18,1872-12-15,PI,Jaicós
782,https://dadosabertos.camara.leg.br/api/v2/depu...,MACEDO,12,17,Joaquim Manoel de Macedo,NaN,M,NaN,NaN,1820-06-24,1882-04-11,RJ,Itaboraí
968,https://dadosabertos.camara.leg.br/api/v2/depu...,ROCHA,16,16,Francisco José da Rocha,NaN,M,NaN,NaN,1832-02-10,NaN,BA,Salvador
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,https://dadosabertos.camara.leg.br/api/v2/depu...,EURICELIA CARDOSO,55,55,EURICELIA MELO CARDOSO,NaN,F,NaN,NaN,1976-04-19,NaN,MA,Monção
7377,https://dadosabertos.camara.leg.br/api/v2/depu...,PROFESSOR PACCO,55,55,MARCOS PACCO RIBEIRO COELHO,NaN,M,NaN,NaN,1974-05-12,NaN,DF,Brasília
7597,https://dadosabertos.camara.leg.br/api/v2/depu...,FREITAS DO PT,55,55,MILNE FREITAS SOUZA,NaN,M,NaN,NaN,1962-05-06,NaN,MA,Barra do Corda
7598,https://dadosabertos.camara.leg.br/api/v2/depu...,WASHINGTON CORAÇÃO VALENTE,55,55,WASHINGTON STECANELA CERQUEIRA,NaN,M,NaN,NaN,1975-04-01,NaN,DF,Brasília


In [15]:
df_nome_genero = df_nome_genero[['nome', 'siglaSexo', 'ufNascimento', 'dataNascimento']]
df_nome_genero.drop_duplicates(subset='nome', keep='first', inplace=True)
df_genero = df_nome_genero
df_genero

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,nome,siglaSexo,ufNascimento,dataNascimento
221,MACEDO,M,NaN,NaN
376,ROCHA,M,RJ,1812-11-08
2823,SILVIO COSTA,M,PE,1956-12-23
2824,IZALCI LUCAS,M,MG,1956-04-07
2825,DANILO FORTE,M,CE,1958-09-01
...,...,...,...,...
7376,EURICELIA CARDOSO,F,MA,1976-04-19
7377,PROFESSOR PACCO,M,DF,1974-05-12
7597,FREITAS DO PT,M,MA,1962-05-06
7598,WASHINGTON CORAÇÃO VALENTE,M,DF,1975-04-01


In [16]:
from datetime import datetime

datetime.today()
diferenca_em_dias = datetime.today() - pd.to_datetime(df_nome_genero.dataNascimento)
idade = np.array((diferenca_em_dias.dt.days/365).values, dtype='int')
media = idade[idade > 0].mean()

idade[idade < 0] = int(media)
df_nome_genero['idade'] = idade

df_nome_genero

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,nome,siglaSexo,ufNascimento,dataNascimento,idade
221,MACEDO,M,NaN,NaN,56
376,ROCHA,M,RJ,1812-11-08,207
2823,SILVIO COSTA,M,PE,1956-12-23,63
2824,IZALCI LUCAS,M,MG,1956-04-07,63
2825,DANILO FORTE,M,CE,1958-09-01,61
...,...,...,...,...,...
7376,EURICELIA CARDOSO,F,MA,1976-04-19,43
7377,PROFESSOR PACCO,M,DF,1974-05-12,45
7597,FREITAS DO PT,M,MA,1962-05-06,57
7598,WASHINGTON CORAÇÃO VALENTE,M,DF,1975-04-01,44


In [0]:
df_nome_genero = df_nome_genero.sort_values(by='nome', ascending=True)
df_nome_genero = df_nome_genero.set_index('nome')

In [0]:
#pd.concat([gasto, df_nome_genero], axis=1, sort=False)

## AGORA COM MERGE

In [0]:
gasto_merge = gasto

In [21]:
gasto_merge = gasto_merge.reset_index()
gasto_merge = gasto_merge.merge(df_genero, left_on='txNomeParlamentar', right_on='nome')
gasto_merge.columns

Index(['index', 'txNomeParlamentar', 'sgPartido', 'vlrDocumento', 'nome_x',
       'siglaSexo_x', 'ufNascimento_x', 'dataNascimento_x', 'idade_x',
       'nome_y', 'siglaSexo_y', 'ufNascimento_y', 'dataNascimento_y',
       'idade_y'],
      dtype='object')

In [22]:
gasto_merge

,index,txNomeParlamentar,sgPartido,vlrDocumento,nome_x,siglaSexo_x,ufNascimento_x,dataNascimento_x,idade_x,nome_y,siglaSexo_y,ufNascimento_y,dataNascimento_y,idade_y
0,0,ABEL MESQUITA JR.,DEM,531047.71,ABEL MESQUITA JR.,M,RR,1962-03-29,57,ABEL MESQUITA JR.,M,RR,1962-03-29,57
1,1,ADAIL CARNEIRO,PODE,546244.27,ADAIL CARNEIRO,M,CE,1963-07-11,56,ADAIL CARNEIRO,M,CE,1963-07-11,56
2,2,ADALBERTO CAVALCANTI,AVANTE,497902.97,ADALBERTO CAVALCANTI,M,BA,1958-02-10,61,ADALBERTO CAVALCANTI,M,BA,1958-02-10,61
3,3,ADELMO CARNEIRO LEÃO,PT,378203.93,ADELMO CARNEIRO LEÃO,M,MG,1949-05-25,70,ADELMO CARNEIRO LEÃO,M,MG,1949-05-25,70
4,4,ADELSON BARRETO,PR,333038.13,ADELSON BARRETO,M,SE,1964-09-22,55,ADELSON BARRETO,M,SE,1964-09-22,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,549,ZÉ CARLOS,PT,484381.83,ZÉ CARLOS,M,MA,1955-05-26,64,ZÉ CARLOS,M,MA,1955-05-26,64
550,550,ZÉ GERALDO,PT,505529.11,ZÉ GERALDO,M,ES,1961-12-21,58,ZÉ GERALDO,M,ES,1961-12-21,58
551,551,ZÉ SILVA,SOLIDARIEDADE,384928.09,ZÉ SILVA,M,MG,1963-05-11,56,ZÉ SILVA,M,MG,1963-05-11,56
552,552,ÁTILA LINS,PP,437089.95,ÁTILA LINS,M,AM,1950-11-22,69,ÁTILA LINS,M,AM,1950-11-22,69


In [23]:
X = gasto_merge[['siglaSexo_x', 'sgPartido', 'idade_x', 'ufNascimento_x']]
X

,siglaSexo_x,sgPartido,idade_x,ufNascimento_x
0,M,DEM,57,RR
1,M,PODE,56,CE
2,M,AVANTE,61,BA
3,M,PT,70,MG
4,M,PR,55,SE
...,...,...,...,...
549,M,PT,64,MA
550,M,PT,58,ES
551,M,SOLIDARIEDADE,56,MG
552,M,PP,69,AM


In [0]:
y = gasto_merge['vlrDocumento']

In [25]:
y

0      531047.71
1      546244.27
2      497902.97
3      378203.93
4      333038.13
         ...    
549    484381.83
550    505529.11
551    384928.09
552    437089.95
553    452594.23
Name: vlrDocumento, Length: 554, dtype: float64

In [28]:
for n,c in X.items():
    if is_string_dtype(c): 
      X[n] = c.astype('category').cat.as_ordered()
      tamanho = len(X[n].cat.categories)
      X[n] = X[n].cat.rename_categories(range(tamanho))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [29]:
X

,siglaSexo_x,sgPartido,idade_x,ufNascimento_x
0,1,1,57,21
1,1,7,56,5
2,1,0,61,4
3,1,20,70,10
4,1,11,55,24
...,...,...,...,...
549,1,20,64,9
550,1,20,58,7
551,1,24,56,10
552,1,8,69,2


In [30]:
y

0      531047.71
1      546244.27
2      497902.97
3      378203.93
4      333038.13
         ...    
549    484381.83
550    505529.11
551    384928.09
552    437089.95
553    452594.23
Name: vlrDocumento, Length: 554, dtype: float64

Classificador de deputados

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier



In [0]:
f_0 = (18019.76, 340184.95)
f_1 = (340184.95,427597.55)
f_2 = (427597.55,476438.02)
f_3 = (476438.02,618776.41)


In [0]:
y_class = []
for numero in y:
    if numero  > f_0[0] and numero < f_0[1]:
        y_class.append("FAIXA 0")
    elif numero  > f_1[0] and numero < f_1[1]:
        y_class.append("FAIXA 1")
    elif numero  > f_2[0] and numero < f_2[1]:
        y_class.append("FAIXA 2")
    elif numero  > f_3[0] and numero < f_3[1]:
        y_class.append("FAIXA 3")

In [0]:
y_class = np.array(y_class)

In [0]:
x_train, x_test, y_test = train_test_split(x, y_class, test_size=0.33, random_state=42)

In [0]:
def calcula_meu_acerto(predicted, y_test):
    erro = 0
    for y_chapeu, y_real in zip(predicted, y_test):
        if y_chapeu != y_real:
            erro += 1
    return 1 - (erro/len(predicted))

In [73]:
calcula_meu_acerto(y_test , predicted)

0.98